# Import

In [1]:
!pip install openai
from openai import OpenAI
import pandas as pd
import numpy as np
import os
import openai
import json

# Data Preprocessing

In [ ]:
file_path = '/content/fine_tuning_expanded.xlsx'
df = pd.read_excel(file_path)

In [ ]:
output = []

for _, row in df.iterrows():
    conversation = {
        'messages': [
            {'role': 'user', 'content': row['question']},
            {'role': 'assistant', 'content': row['answer']}
        ]
    }
    output.append(conversation)

In [ ]:
# 훈련 json data
file_path = "data_gpt-3.5-turbo_train.jsonl"
with open(file_path, "w", encoding="utf-8") as file:
    for message in output:
        json.dump(message, file, ensure_ascii=False)
        file.write("\n")
print("훈련 jsonl 완")

훈련 jsonl 완


#Train

In [ ]:
api_key = ""

client = OpenAI(api_key=api_key)

In [ ]:
# OpenAI API 키 설정
api_key = ""

client = OpenAI(api_key=api_key)

# 훈련 데이터 jsonl 파일 업로드
file_train=client.files.create(
  file=open("data_gpt-3.5-turbo_train.jsonl", "rb"),
  purpose="fine-tune"
)
print(file_train.id)

FileNotFoundError: [Errno 2] No such file or directory: 'data_gpt-3.5-turbo_train.jsonl'

In [ ]:
# Fine-Tuning 시작
client.fine_tuning.jobs.create(
  training_file=file_train.id,
  model="gpt-4o-mini-2024-07-18",
  # model = 'gpt-3.5-turbo',
  hyperparameters={
    "n_epochs":10,
    "learning_rate_multiplier":1,
    "batch_size":4
  }
)

# Test

In [7]:
# ChatGPT 기본 모델 호출 함수
def ask_chatgpt1(question):

    # api key 지정
    client = OpenAI(api_key = api_key)

    # # API를 사용하여 'gpt-3.5-turbo' 모델로부터 응답을 생성합니다.
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},  # 기본 역할 부여
            {"role": "user", "content": question},                          # 질문
        ]
    )

    return response.choices[0].message.content

# 튜닝된 모델 호출 함수
def tunning_model(question):

    # api key 지정
    client = OpenAI(api_key = api_key)

    # # API를 사용하여 'gpt-3.5-turbo' 모델로부터 응답을 생성합니다.
    response = client.chat.completions.create(
        model="ft:gpt-4o-mini-2024-07-18:aivle::AmdMMDVB",
       # model = 'gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},  # 기본 역할 부여
            {"role": "user", "content": question},                          # 질문
        ]
    )

    return response.choices[0].message.content

In [20]:
question = "상주 은모래비치 포토존을 알려줘"

# 모델 호출
result1 = ask_chatgpt1(question)
result2 = tunning_model(question)

# 결과 비교 DataFrame 생성
df2 = pd.DataFrame({
    "질문": [question],
    "ChatGPT 기본 모델 응답": [result1],
    "튜닝된 모델 응답": [result2]
})

In [21]:
# 데이터프레임 출력 옵션 설정
pd.options.display.max_colwidth = None  # 각 열의 최대 너비를 제한 없이 설정
pd.options.display.width = 0  # 출력 폭 제한 해제

display(df2)

,질문,ChatGPT 기본 모델 응답,튜닝된 모델 응답
0,상주 은모래비치 포토존을 알려줘,"죄송하지만, 제가 알고 있는 정보에는 ""상주 은모래비치 포토존""에 대한 구체적인 정보가 없습니다. 원하시는 정보를 찾으시기 위해 관련 사이트나 지역 관광 안내소 등을 이용하시거나, 해당 명칭을 포함하여 좀 더 구체적인 검색어로 온라인 검색을 시도해 보시는 것을 권해드립니다. 부디 원하시는 정보를 빠르게 찾으시길 바랍니다. 필요하신 정보가 있으면 다시 물어봐 주세요. 도움이 필요하시면 말씀해주세요.",상주 은모래비치에 '은빛 물결' 조형물이 있습니다. 자세한 내용은 추가 문의 바랍니다.
